In [ ]:
# https://pytorch.kr/hub/nvidia_deeplearningexamples_tacotron2/
# 타코트론2와 웨이브글로우를 pytorchhub에서 가져와서 쓰는 법
# api 가져다 쓰는 느낌임 # 사실 영어라서 잘 안쓸듯?
# 그냥 구조만 알아볼려고 해봄
# 텍스트 -> 전처리 -> melspectrogram -> 소리

import torch
# 오디오파일 관리를 위해 
import numpy as np
from scipy.io.wavfile import write

In [ ]:
!pip install unidecode 

     |████████████████████████████████| 235 kB 5.4 MB/s 


In [ ]:
# colab사용시 gpu 키고 실행
# 사전학습 된 tacotron2 모델

waveglow = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_waveglow', model_math='fp16')
waveglow = waveglow.remove_weightnorm(waveglow)
waveglow = waveglow.to('cuda')
waveglow.eval()

Downloading: "https://github.com/NVIDIA/DeepLearningExamples/archive/torchhub.zip" to /root/.cache/torch/hub/torchhub.zip


WaveGlow(
  (upsample): ConvTranspose1d(80, 80, kernel_size=(1024,), stride=(256,))
  (WN): ModuleList(
    (0): WN(
      (in_layers): ModuleList(
        (0): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (2): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
        (3): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
        (4): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(16,), dilation=(16,))
        (5): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(32,), dilation=(32,))
        (6): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(64,), dilation=(64,))
        (7): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(128,), dilation=(128,))
      )
      (res_skip_layers): ModuleList(
        (0): Conv1d(512, 1024, kernel_size=(1,), stride=(1,))
        (1): Conv1d(51

In [ ]:
# 사전학습 된 tacotron2 모델

tacotron2 = torch.hub.load('NVIDIA/DeepLearningExamples', 'nvidia_tacotron2', model_math='fp16')
tacotron2 = tacotron2.to('cuda')
tacotron2.eval()

Downloading: "https://github.com/NVIDIA/DeepLearningExamples/archive/master.zip" to /root/.cache/torch/hub/master.zip


Tacotron2(
  (embedding): Embedding(148, 512)
  (encoder): Encoder(
    (convolutions): ModuleList(
      (0): Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (2): Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (lstm): LSTM(512, 256, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (prenet): Prenet(
      (layers): ModuleList(
        (0): LinearNorm(
          (lin

In [ ]:
sample_text = 'hello tacotron2! hello waveglow!'
sample_text_list = ['hello tacotron2! hello waveglow!', 'hi my name is Jin', 'man in black jacket and jeans is standing on sidewalk']

In [ ]:
# 전처리

# sequence = np.array(tacotron2.text_to_sequence(sample_text, ['english_cleaners']))[None, :]
# sequence = torch.from_numpy(sequence).to(device='cuda', dtype=torch.int64)

# 코랩에 unidecode가 안껄려있었어 에라이
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tts_utils')
sequences, lengths = utils.prepare_input_sequence([sample_text])



Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [ ]:
sequences, lengths = utils.prepare_input_sequence([sample_text_list[2]])

In [ ]:
# 돌려
with torch.no_grad():
    # Mel spectogram
    mel, _, _ = tacotron2.infer(sequences, lengths)
    # inference
    audio = waveglow.infer(mel)

audio_numpy = audio[0].data.cpu().numpy()
rate = 22050

write('audio.wav', rate, audio_numpy)

In [ ]:
# 확인
print(sample_text_list[2])
from IPython.display import Audio
Audio(audio_numpy, rate=rate)

man in black jacket and jeans is standing on sidewalk
